In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\diamond_price_predication_2'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig: 
    root_dir: Path
    test_data_path: Path
    model_: Path
    metric_file: Path
    all_parmas: dict
    target_column: str

In [29]:
from diamond_price_predication.constant import *
from diamond_price_predication.utils.common import read_yaml, create_dicectories, save_json

In [20]:
class ConfigurationManger: 
    def __init__(self, config_file_path= CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH, 
                 schema_file_path=SCHEMA_FILE_PATH): 
        self.config=read_yaml(config_file_path)
        self.schema=read_yaml(schema_file_path)
        self.params=read_yaml(params_file_path)
        
        
        create_dicectories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self)-> ModelEvaluationConfig: 
            
        config=self.config.model_evaluation
        schema=self.schema.TARGET_COLUMN
        params=self.params.ElasticNet
            
        create_dicectories([config.root_dir])
            
            
        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir, 
            test_data_path=config.test_data_path, 
            model_=config.model_path,
            metric_file=config.metric_file_path,
            all_parmas=params, 
            target_column=schema.name)
        return model_evaluation_config
            
            
            
            
            

In [21]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
import joblib
from diamond_price_predication import logger

In [38]:
class ModelEvaluation: 
    def __init__(self, config= ModelEvaluationConfig): 
        self.config=config
        
        
    def Evaluation(self, actual, predicted):
        
        mse=mean_squared_error(actual , predicted)
        rms=np.sqrt(mean_squared_error(actual, predicted))
        r2score=r2_score(actual, predicted)
        
        return mse, rms, r2score 
    
    def save_result(self): 
        
        model= joblib.load(self.config.model_)  
        logger.info(f"model loaded sucessfully: {model}")
        
        test_data=pd.read_csv(self.config.test_data_path)
        logger.info(f"Test data sucessfullt {test_data.head()}")
        
        x_test=test_data.drop([self.config.target_column], axis=1)
        y_test=test_data[[self.config.target_column]]
        
        y_predict_=model.predict(x_test)
        
        (mse, rms, r2score)=self.Evaluation(y_test, y_predict_)
        
        # logger.info(f"Mean squared error: {mse}, root_mean_error: {rms}, root_mean: {r2score}")
        
        
        score={"MSE": mse, "RMS": rms, "r2score": r2score}
        
        save_json(path=Path(self.config.metric_file), data=score)
        
        logger.info(f"Metric score saved {score}")
        
                
        

In [39]:
try: 
    configmanager=ConfigurationManger()
    model_evaluation_config=configmanager.get_model_evaluation_config() 
    modelevaluation=ModelEvaluation(model_evaluation_config)
    # modelevaluation.Evaluation()
    modelevaluation.save_result()
except Exception as e:
    raise e

[2023-12-13 15:19:41,111 : INFO: common : Yaml file read config\config.yaml]
[2023-12-13 15:19:41,123 : INFO: common : Yaml file read schema.yaml]
[2023-12-13 15:19:41,196 : INFO: common : Yaml file read params.yaml]
[2023-12-13 15:19:41,201 : INFO: common : Directories file is creted artifacts]
[2023-12-13 15:19:41,205 : INFO: common : Directories file is creted artifacts/data_evaluation]
[2023-12-13 15:19:41,213 : INFO: 1180672315 : model loaded sucessfully: ElasticNet(alpha=0.1, l1_ratio=0.2)]


[2023-12-13 15:19:41,391 : INFO: 1180672315 : Test data sucessfullt    carat  cut  color  clarity  depth  table     x     y     z  price
0   0.33    5      5        5   62.2   55.0  4.43  4.46  2.77    579
1   1.04    3      4        5   63.2   56.0  6.47  6.38  4.07   6817
2   1.39    5      6        5   60.7   56.0  7.25  7.30  4.42   7929
3   0.31    5      3        8   61.9   55.0  4.36  4.39  2.71    914
4   0.73    5      2        4   61.1   57.0  5.79  5.84  3.55   2723]
[2023-12-13 15:19:41,452 : INFO: common : Json file saved at artifacts\data_evaluation\metrics.json]
[2023-12-13 15:19:41,476 : INFO: 1180672315 : Metric score saved {'MSE': 2198764.98919526, 'RMS': 1482.8233169178518, 'r2score': 0.8660832290728853}]
